<h1><center>Pharmaceutical-Sales-prediction </center></h1>

In [138]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
# import necessary libraries
import pandas as pd
import numpy as np
import dvc.api
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import mlflow
# To ignore any warnings
import warnings                        
warnings.filterwarnings("ignore")
import os
import logging


In [140]:
#Load training data 
path = 'data/train.csv'
repo = '/home/bethelhem/Pharmaceutical-Sales-prediction'
version = 'train_v1'
data_url2 = dvc.api.get_url(
    path = path,
    repo = repo,
    rev=version
    )
try:
    train_data=pd.read_csv(data_url2)  
except FileNotFoundError:
        print("file not found")
train_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [141]:
#load test data
path = 'data/test.csv'
repo = '/home/bethelhem/Pharmaceutical-Sales-prediction'
version = 'test_v1'
data_url3 = dvc.api.get_url(
    path = path,
    repo = repo,
    rev=version
    )
try:
    test_data=pd.read_csv(data_url3) 

except FileNotFoundError:
        print("file not found")
test_data.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [142]:
#load store data
path = 'data/store.csv'
repo = '/home/bethelhem/Pharmaceutical-Sales-prediction'
version = 'store_v1'
data_url = dvc.api.get_url(
    path = path,
    repo = repo,
    rev=version
    )
try:
    store_data=pd.read_csv(data_url)
except FileNotFoundError:
        print("file not found")
store_data.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


#### Glance the data

In [143]:
print("shape of the  store data: ",store_data.shape)
print(store_data.info())

shape of the  store data:  (1115, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB
None


In [144]:
print("shape of the  train data: ",train_data.shape)
print(train_data.nunique())

shape of the  train data:  (1017209, 9)
Store             1115
DayOfWeek            7
Date               942
Sales            21734
Customers         4086
Open                 2
Promo                2
StateHoliday         5
SchoolHoliday        2
dtype: int64


### Data cleaning

In [145]:
#data extraction
try:
    train_data['Date'] = pd.to_datetime(train_data['Date'])
    train_data['Year'] = train_data['Date'].dt.year
    train_data['Month'] = train_data['Date'].dt.month
    train_data['Day'] = train_data['Date'].dt.day
    train_data['WeekOfYear'] = train_data['Date'].dt.weekofyear
except KeyError:
        print("Column couldn't be found")
        
#Coping dataframe for later plots
train_data_copy=train_data.copy() #for later plots


In [146]:
train_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear
0,1,5,2015-07-31,5263,555,1,1,0,1,2015,7,31,31
1,2,5,2015-07-31,6064,625,1,1,0,1,2015,7,31,31
2,3,5,2015-07-31,8314,821,1,1,0,1,2015,7,31,31
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015,7,31,31
4,5,5,2015-07-31,4822,559,1,1,0,1,2015,7,31,31


In [147]:
#checking for missing values in training data
train_data.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
Year             0
Month            0
Day              0
WeekOfYear       0
dtype: int64


- As shown above the training data has no missing values.

In [148]:
#checking for missing values in test data
test_data.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64


-  The test data has a missing value for open variable , we impute it using the mode.

In [149]:
# impute the missing value in test data
test_data['Open'].fillna(test_data['Open'].mode, inplace = True)
test_data.isnull().values.any()

False

In [ ]:
# closed stores
train_data[(train_data.Open == 0) & (train_data.Sales == 0)].count()

In [ ]:
print(train_data.shape)
closed_stores = train_data[(train_data['Open'] == 0) & (train_data['Sales'] == 0)].index
#To avoid any biased forecasts we will drop these values.
train_data.drop(closed_stores,inplace = True)
print(train_data.shape)


In [ ]:
open_no_sale = train_data[(train_data["Open"] != 0) & (train_data['Sales'] == 0)].index
train_data[(train_data["Open"] != 0) & (train_data['Sales'] == 0)]['Date'].nunique()

- There are open stores with no sales.We can presume that external variables played a role. There're only 49 days in the data.

In [ ]:
train_data.drop(open_no_sale,inplace = True)
train_data.shape

In [ ]:
# Stores with maximum sale count
train_data.groupby('Store')['Sales','Customers'].sum().sort_values('Sales', ascending=False).head()

In [ ]:
store_exc = train_data[(train_data["StateHoliday"] != "o") & (train_data["Sales"] != 0)]["Store"].nunique()
print(f"There are {store_exc} stores that have opened during state holidays.")

In [ ]:
#check for missing values in store dataframe
store_data.isnull().sum()

- We need to deal with a couple of variables that have missing values.

In [ ]:
#fill missing values in CompetitionDistance with median value
store_data['CompetitionDistance'].fillna(store_data['CompetitionDistance'].median(), inplace = True)

In [ ]:
#fill missing values with mode
store_data['CompetitionOpenSinceMonth'].fillna(store_data['CompetitionOpenSinceMonth'].mode, inplace = True)
store_data['CompetitionOpenSinceYear'].fillna(store_data['CompetitionOpenSinceYear'].mode, inplace = True)
store_data['Promo2SinceWeek'].fillna(store_data['Promo2SinceWeek'].mode, inplace = True)
store_data['Promo2SinceYear'].fillna(store_data['Promo2SinceYear'].mode, inplace = True)
store_data['PromoInterval'].fillna(store_data['PromoInterval'].mode, inplace = True)
store_data.isnull().values.any()

- Let us take a closer look at the various StoreType levels and how the main metric Sales is distributed among them.

In [ ]:
print("Joining train set with an additional store information.")
train_store = train_data.merge(store_data,on=["Store"],how="inner")


In [ ]:
train_store.shape

In [ ]:
train_store.groupby('StoreType')['Sales'].describe()

- Among all StoreTypes, StoreType B has the greatest average of Sales, although we have significantly less data for it.StoreType D has the least average of Sales.

In [ ]:
#overall sum of Sales and Customers to see which StoreType is the most selling and crowded one
train_store.groupby('StoreType')['Customers', 'Sales'].sum()

 - StoreType A is the most selling and crowded one.

## Correlation matrix

In [ ]:

corrMatrix= train_store.drop('Open', axis = 1).corr()
f, ax = plt.subplots(figsize=(16, 14))
res=sns.heatmap(corrMatrix,cmap="Blues",annot=True,fmt='.4f',annot_kws={"size":12})
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 12)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 12)
plt.show()

- As shown in the figure, the dark Blue squares represents more closely correlated data. **sales and customers data are highly correlated**.

In [ ]:
sns.set(style="whitegrid")
ax = sns.barplot(x="Month", y="Sales", data=train_store)
plt.title('Monthly Sales data\n', size=15, fontweight='bold')

In [ ]:
#check seasonality for the given store
def seasonality(store):
    sales = train_data[train_data.Store == store]
    sales = sales.sort_values(by = "Date",ascending= False)
    sns.set(rc = {'figure.figsize':(20,8)})
    sns.lineplot(x = sales["Date"], y = sales["Sales"], color= "green")
    plt.xticks(rotation=90)
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Sales', fontsize=16)
    plt.title(f'Seasonality of store {store}\n', size=20, fontweight='bold')
    plt.show()

In [ ]:
seasonality(1)

In [ ]:
#check seasonality for 2013,2014 and 2015 for a given store
def seasonality_year(store):
    sale = train_data[train_data.Store == store]
    sale = sale[(sale.Year == 2015) | (sale.Year == 2014) | (sale.Year == 2013)]
    sale = sale.sort_values(by = "Date",ascending= False)
    sns.set(rc = {'figure.figsize':(20,8)})
    sns.lineplot(x = sale["Date"], y = sale["Sales"], color= "green")
    plt.xticks(rotation=90)
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Sales', fontsize=16)
    plt.title(f'Seasonality of store {store} from 2013-2015 \n', size=20, fontweight='bold')
    plt.show()

In [ ]:
seasonality_year(1)

In [ ]:
def plot_counts(df:pd.DataFrame, column1:str) -> None:
    plt.figure(figsize=(4, 2))
    sns.countplot(data=df, x=column1)
    plt.title(f'Distribution of {column1}\n', size=13, fontweight='bold')
    plt.show()

In [ ]:
promo = pd.DataFrame()
promo['train promo'] = train_data['Promo']
promo['test promo'] = test_data['Promo']



In [ ]:
#distribution of train promo
plot_counts(promo, 'train promo')


In [ ]:
#distribution of test promo
plot_counts(promo, 'test promo')

In [ ]:
#sales behavior before, during and after holidays
pre_holiday = train_data[(train_data["Date"]> '2013-12-09') & (train_data["Date"]< '2013-12-22')  ]

during_holiday=train_data[(train_data["Date"]> '2013-12-22') & (train_data["Date"]< '2013-12-31')  ]

post_holiday = train_data[(train_data["Date"]> '2013-12-31') & (train_data["Date"]< '2014-01-15')  ]

In [ ]:
sns.lineplot(x = pre_holiday["Date"] , y = pre_holiday["Sales"] )
sns.lineplot(x = during_holiday["Date"] , y = during_holiday["Sales"] )
sns.lineplot(x = post_holiday["Date"] , y = post_holiday["Sales"] )
plt.xticks(rotation=90)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Sales', fontsize=16)
plt.title(f'Sales behavior before, during and after Christmas \n', size=20, fontweight='bold')
plt.show()

- As demonstrated in the figure, purchases prior to the holiday are large, while purchases during the holiday are relatively low.

In [ ]:
#Customers before, during and after Christmas
sns.lineplot(x = pre_holiday["Date"] , y = pre_holiday["Customers"] )
sns.lineplot(x = during_holiday["Date"] , y = during_holiday["Customers"] )
sns.lineplot(x = post_holiday["Date"] , y = post_holiday["Customers"] )
plt.xticks(rotation=90)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Customers', fontsize=16)
plt.title(f'Customers before, during and after Christmas \n', size=20, fontweight='bold')
plt.show()

### Trends of customer behavior during store open and closing times


In [ ]:
sns.factorplot(data = train_data_copy, x = 'Month', hue = 'Open',
               y = "Customers", 
               col = 'Open')
 
plt.title(f'Customers behavior during store open and closing times \n\n ', size=20)       

#### Which stores are opened on all weekdays?

In [ ]:
#customers trends
g=sns.factorplot(data = train_store, x = 'Month', y = "Sales", hue = 'StoreType',
               col = 'StoreType', # per store type in cols
               row = 'DayOfWeek', # per day of week in the stores
               ) 
g.set_titles(size=18)


- We can see that stores of StoreType C are all closed on Sundays, whereas others are most of the time opened.StoreType D stores are only closed on Sundays from October to December.

#### Effect of Assorment type on sales

In [ ]:
sns.factorplot(data = train_store, x = 'Month', y = "Sales",hue = 'Assortment',
               col = 'Assortment') 
plt.title("Effect of Assorment type on sales\n\n ", size=20)

- Assortment type b has significant sales volume, and  the trend that assortment b use is better than others.

#### Effect of promotion on sales and number of customers

In [ ]:
#promo and sales
sns.factorplot(data = train_store, x = 'Month', y = "Sales",hue = 'Promo',col = 'Promo') 
plt.title(f'Effect of promotion on sales\n\n ', size=20)

In [ ]:
#promo and customers
sns.factorplot(data = train_store, x = 'Month', y = "Customers",hue = 'Promo',col = 'Promo')
plt.title(f'Effect of promotion on number of customers\n\n ', size=20)

In [ ]:
#Could the promos be deployed in more effective ways? Which stores should promos be deployed in?
#customers trends
sns.factorplot(data = train_store, x = 'Month', y = "Customers", 
               col = 'Promo', # per store type in cols
               hue = 'StoreType',
               row = 'StoreType', # per promo in the store in rows
               ) 

#### Effect of Distance to the next competitor on sales

In [ ]:
comp_store = train_store.groupby("Store").agg({'CompetitionDistance':'max','Sales':'sum'})

In [ ]:
sns.lineplot(x = 'CompetitionDistance',y = "Sales",
             data = comp_store)

- The more stores that are in close proximity to each other, the more sales they make.
- This suggests that where other stores are within 10 kilometers, sales are higher.


## Prophet
- Implements a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

In [ ]:
def preprocess_(df):
    # remove closed stores and those with no sales
    df = df[(df["Open"] != 0) & (df['Sales'] != 0)]

    # sales for the store number 1 (StoreType C)
    sales = df[df.Store == 1].loc[:, ['Date', 'Sales']]

    # reverse to the order: from 2013 to 2015
    sales = sales.sort_index(ascending = False)

    # to datetime64
    sales['Date'] = pd.DatetimeIndex(sales['Date'])
    sales = sales.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
    # plot daily sales
    ax = sales.set_index('ds').plot(figsize = (12, 4), color = "red")
    ax.set_ylabel('Daily Number of Sales')
    ax.set_xlabel('Date')
    plt.show()
    return sales 

In [ ]:
# modeling holidays
def modeling_holiday(df):
    # create holidays dataframe
    state_dates = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].loc[:, 'Date'].values
    school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values

    state = pd.DataFrame({'holiday': 'state_holiday',
                          'ds': pd.to_datetime(state_dates)})
    school = pd.DataFrame({'holiday': 'school_holiday',
                          'ds': pd.to_datetime(school_dates)})

    holidays = pd.concat((state, school))
    return holidays

In [ ]:
from fbprophet import Prophet
import pickle
import datetime
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics


class Fb_prophet:
    def __init__(self,train,holidays):
        self.future_dates = None
        self.forecast = None
        
        self.model = Prophet(interval_width = 0.95, holidays = holidays)
        self.model.fit(train)

          # dataframe that extends into future 6 weeks 
    def getFuture_dates(self,week):
        self.future_dates = self.model.make_future_dataframe(periods = int(week)*7)
        return self.future_dates
    def get_predicate(self):
        # predictions
        self.forecast = self.model.predict(self.future_dates)
       
        self.forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        fc = self.forecast[['ds', 'yhat']].rename(columns = {'ds': 'Date', 'yhat': 'Forecast_Sales'})
        
        return fc
    def eff_modlel(self):
        # visualizing predicions
        fig1 = self.model.plot(self.forecast)
        fig1.savefig('forecastS.png')
    def get_components(self):
        fig2=self.model.plot_components(self.forecast)
        fig2.savefig('forecastComponents.png')
    def get_performanceMetrics(self):
        cross_validation_results = cross_validation(self.model, initial='370 days', period=3*7, horizon='70 days')
        print(cross_validation_results)

        performance_metrics_results = performance_metrics(cross_validation_results)
        return performance_metrics_results
    def save_model(self):
        now = datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S-%f')[:-3]
        # Saving model to disk
        filename = now + '.pkl'
        pickle.dump(self.model, open(filename, 'wb'))
        return filename

In [ ]:

sales = preprocess_(train_data)

In [ ]:
holidays = modeling_holiday(train_data)

In [ ]:
prophet = Fb_prophet(sales,holidays)

In [ ]:
week=3 #no of weeks we want to forcast
fd = prophet.getFuture_dates(week)
predict =prophet.get_predicate()


In [ ]:
sns.lineplot(x=predict['Date'],y=predict['Forecast_Sales'],data=predict)

In [ ]:
prophet.get_performanceMetrics()


In [ ]:
data_f = pd.DataFrame()
start_date= '2016-11-11'
end_date= '2016-12-25'
data_f['ds'] = pd.date_range(start_date, periods = 48, freq ='D') 

p = prophet.model.predict(data_f)
fc = p[['ds', 'yhat']].rename(columns = {'ds': 'Date', 'yhat': 'Forecast_Sales'})
sns.lineplot(x=fc['Date'],y=fc['Forecast_Sales'],data=fc)

In [ ]:
prophet.eff_modlel()
filename = prophet.save_model()

In [ ]:
prophet.get_components()